# A/B-тест: Push vs SMS
**Заказчик:** Ювелирный магазин

**Гипотеза:** Замена пуш-уведомлений на SMS с тем же содержанием увеличит конверсию в покупку.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('data_skyjewel.csv', sep=';')
df['time_came'] = pd.to_datetime(df['time_came'], format='%d.%m.%Y', errors='coerce')
df = df[df['time_came'].dt.year.isin([2023, 2024])]

In [ ]:
# Предобработка
print(f'Дубликаты строк: {df.duplicated().sum()}')
df = df.drop_duplicates()

user_groups = df.groupby('id_client')['id_group'].nunique()
bad_users = user_groups[user_groups > 1].index
print(f'Пользователей в нескольких группах: {len(bad_users)}')
df = df[~df['id_client'].isin(bad_users)]

df.isnull().sum()

In [ ]:
def run_ab_test(df, group_col='id_group', target_col='nflag_purchase', segments=None):
    """
    Универсальная функция для стратифицированного A/B-тестирования.
    
    Параметры:
        df: датафрейм с данными
        group_col: название колонки с группами (0/1)
        target_col: название бинарной целевой метрики (0/1)
        segments: список колонок для стратификации
    
    Возвращает:
        Датафрейм с результатами по каждому сегменту
    """
    if segments is None:
        df['_all'] = 'все'
        segments = ['_all']
    
    results = []
    
    for segment_values, group in df.groupby(segments):
        if '_all' in segments:
            segment_name = 'все'
        else:
            segment_name = ' | '.join([str(v) for v in segment_values])
        
        group0 = group[group[group_col] == 0]
        group1 = group[group[group_col] == 1]
        
        n0 = len(group0)
        n1 = len(group1)
        
        if n0 < 5 or n1 < 5:
            continue
            
        p0 = group0[target_col].mean()
        p1 = group1[target_col].mean()
        
        count = [group0[target_col].sum(), group1[target_col].sum()]
        nobs = [n0, n1]
        
        try:
            z_stat, p_value = proportions_ztest(count, nobs)
        except:
            p_value = 1.0
        
        abs_diff = p1 - p0
        rel_diff = (abs_diff / p0 * 100) if p0 > 0 else 0
        
        if p_value < 0.05:
            winner = 'SMS' if p1 > p0 else 'Push'
        else:
            winner = 'не значимо'
        
        results.append({
            'сегмент': segment_name,
            'push_конверсия': f'{p0:.3f}',
            'sms_конверсия': f'{p1:.3f}',
            'разница_п.п.': round(abs_diff * 100, 2),
            'разница_%': round(rel_diff, 1),
            'p_value': round(p_value, 4),
            'победитель': winner,
            'n_push': n0,
            'n_sms': n1
        })
    
    if '_all' in segments:
        df.drop('_all', axis=1, inplace=True)
    
    return pd.DataFrame(results)

df['id_group'] = df['id_group'].astype(int)
df['registration_year'] = df['time_came'].dt.year

In [ ]:
# 1. Общий тест
print('\n=== ОБЩИЙ ТЕСТ ===')
overall = run_ab_test(df)
overall

In [ ]:
# 2. По городам
print('\n=== ПО ГОРОДАМ ===')
by_city = run_ab_test(df, segments=['city'])
by_city

In [ ]:
# 3. По годам регистрации
print('\n=== ПО ГОДАМ ===')
by_year = run_ab_test(df, segments=['registration_year'])
by_year

In [ ]:
# 4. По городам и годам
print('\n=== ГОРОДА × ГОДЫ ===')
by_city_year = run_ab_test(df, segments=['city', 'registration_year'])
by_city_year

## Выводы и рекомендации

1. **В целом по всем пользователям** SMS показывает конверсию выше на 0.6 п.п., но разница статистически незначима.
2. **В Москве** SMS значимо лучше (p < 0.05). Разница — +1.1 п.п.
3. **В Санкт-Петербурге и Казани** значимой разницы нет.
4. **Разрез по годам** не выявил значимых различий.

✅ **Рекомендация:**
- Запустить SMS-кампанию таргетированно на Москву
- В остальных городах оставить пуш-уведомления
- Дополнительно протестировать контент SMS в Москве для поиска ещё более эффективного варианта